In [ ]:
import os, geopandas, pandas
import country_converter as coco
from pyproj import CRS

In [ ]:
current_folder = globals()['_dh'][0]
rootdir = os.path.dirname(os.path.dirname(current_folder))
indir_inter = os.path.join(rootdir, '_2_intermediate', 'data')
indir_prepro = os.path.join(rootdir, '_1_preprocessing', 'data', 'raw', 'gis')
outdir = os.path.join(rootdir, '_3_figures_tables', 'data')

In [ ]:
wgs84_crs = CRS.from_string('EPSG:4326') # WGS 1984
wcea_crs = CRS.from_string('esri:54034') # World cylindical equal area projection

In [ ]:
admin_gdf = geopandas.read_file(os.path.join(indir_prepro, 'districts_for_plotting', 'districts_for_plotting.shp'))[['iso', 'district', 'geometry']]
df=pandas.read_csv(os.path.join(indir_inter,'pop_n_land_size_of_districts.csv'))

In [ ]:
df

In [ ]:
admin_gdf

In [ ]:
admin_gdf = admin_gdf.rename(columns={'iso': 'country'})

In [ ]:
admin_gdf['country'] = coco.convert(names = admin_gdf['country'], to = 'name')

In [ ]:
admin_gdf = admin_gdf.to_crs(crs=wcea_crs)

In [ ]:
admin_gdf["area"] = admin_gdf['geometry'].area/ 10**6
admin_gdf

In [ ]:
ids = admin_gdf['country'].unique()
df_ids = pandas.DataFrame(ids)
dfs1=[]
dfs2=[]
dfs3=[]
for id in ids:
    #print(id)
    df1=admin_gdf.loc[admin_gdf['country']==id, 'area'].mean()
    df2=admin_gdf.loc[admin_gdf['country']==id, 'area'].median()
    df3=admin_gdf.loc[admin_gdf['country']==id, 'area'].std()
    #print(df1)
    dfs1.append(df1)
    dfs2.append(df2)
    dfs3.append(df3)

In [ ]:
df_ma = pandas.DataFrame(dfs1)
df_meda = pandas.DataFrame(dfs2)
df_sda = pandas.DataFrame(dfs3)

In [ ]:
df_master=pandas.concat([df_ids,df_ma,df_meda,df_sda], axis=1, ignore_index=True)
df_master.columns=['country','mean_area','median_area','std_are']
df_master

In [ ]:
master=pandas.merge(admin_gdf,df_master, how='left', left_on='country', right_on='country')
master

In [ ]:
master=pandas.merge(master,df, how='left', left_on='country', right_on='country')
master

In [ ]:
master1=master.drop_duplicates(subset=['country'])
master2=master1.drop('geometry',axis=1)
master2